In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
file_path = 'heart.csv' 
df = pd.read_csv(file_path)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [4]:
df['age_category'] = pd.cut(df['age'], bins=[0, 30, 50, 70, 100], labels=['0-30', '31-50', '51-70', '71+'])
df['chol_age_ratio'] = df['chol'] / df['age']
df['bp_chol_ratio'] = df['trestbps'] / df['chol']
df['hr_recovery'] = 220 - df['age'] - df['thalach']

In [5]:
df = pd.get_dummies(df, columns=['age_category'], drop_first=True)

In [6]:
X = df.drop('target', axis=1)
y = df['target']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)  # Retain 95% variance
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
importances = model.feature_importances_

important_features = pd.Series(importances).sort_values(ascending=False)
print("Top features based on importance:\n", important_features.head(10))

top_features = important_features.index[:10]
X_top_pca = X_pca[:, top_features]

X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(X_top_pca, y, test_size=0.3, random_state=42)

model_top = RandomForestClassifier(random_state=42)
model_top.fit(X_train_top, y_train_top)

y_pred_top = model_top.predict(X_test_top)
accuracy = accuracy_score(y_test_top, y_pred_top)
report = classification_report(y_test_top, y_pred_top)
conf_matrix = confusion_matrix(y_test_top, y_pred_top)

Top features based on importance:
 0     0.242152
1     0.127078
6     0.070498
4     0.067727
3     0.066990
10    0.062996
5     0.060752
9     0.057451
2     0.057021
12    0.048617
dtype: float64


In [8]:
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9902597402597403
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       159
           1       1.00      0.98      0.99       149

    accuracy                           0.99       308
   macro avg       0.99      0.99      0.99       308
weighted avg       0.99      0.99      0.99       308

Confusion Matrix:
 [[159   0]
 [  3 146]]
